In [ ]:
import os
import time
import matplotlib.pyplot as plt
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision

from src.dataset_loaders.download_openimages import OpenImagesLoader
from tqdm import tqdm, tqdm_notebook


In [2]:

# Device Configuration:
if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'

print(f"Device being used: {device}")

Device being used: cuda


In [ ]:
import os
import json
import torchvision.transforms.v2 as transforms
from openimages.download import download_dataset
import random
import shutil
import torch
from torch.utils.data import DataLoader
from PIL import Image as PILImage
from torchvision import tv_tensors
from torchvision.transforms.v2 import Resize
from torchvision.ops import box_convert
from PIL import Image

class DETRDatasetProcessor():

    def __init__(self, random_seed = 101, batch_size = 128, perc_keep = 1.0, num_images_per_class=500):
        self.data_dir = os.path.join("data", "openimages")  # Directory in which dataset resides
        self.random_seed = random_seed
        self.batch_size = batch_size
        self.perc_keep = perc_keep  # Percentage of dataset to be kept (number between 0 and 1)
        self.num_images_per_class = num_images_per_class

        self.transforms_all = transforms.Compose(
            [
                Resize((512, 512)),
            ]
        )

        self.transforms_img = transforms.Compose(
            [
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # ImageNet's normalization statistics
            ]
        )

        self.classes = [
            "Hot dog", "French fries", "Waffle", "Pancake", "Burrito", "Pretzel",
            "Popcorn", "Cookie", "Muffin", "Ice cream", "Cake", "Candy",
            "Guacamole", "Apple", "Grape", "Common fig", "Pear",
            "Strawberry", "Tomato", "Lemon", "Banana", "Orange", "Peach", "Mango",
            "Pineapple", "Grapefruit", "Pomegranate", "Watermelon", "Cantaloupe",
            "Egg (Food)", "Bagel", "Bread", "Doughnut", "Croissant",
            "Tart", "Mushroom", "Pasta", "Pizza", "Squid",
            "Oyster", "Lobster", "Shrimp", "Crab", "Taco", "Cooking spray",
            "Cucumber", "Radish", "Artichoke", "Potato", "Garden Asparagus",
            "Pumpkin", "Zucchini", "Cabbage", "Carrot", "Salad",
            "Broccoli", "Bell pepper", "Winter melon", "Honeycomb",
            "Hamburger", "Submarine sandwich", "Cheese", "Milk", "Sushi"
        ]


        self.category_mapping = {i: {"id": i + 1, "name": name.lower()} for i, name in enumerate(self.classes)}
        self.class_2_index = {entry["name"]: entry["id"] for entry in self.category_mapping.values()}

        self.train_dir = os.path.join(self.data_dir, "train") # Directory in which train dataset resides
        self.val_dir = os.path.join(self.data_dir, "val") # Directory in which validation dataset resides
        self.test_dir = os.path.join(self.data_dir, "test") # Directory in which test dataset resides

        self.train_red_dir = os.path.join(self.data_dir, "train_red") # Directory in which reduced train dataset resides
        self.val_red_dir = os.path.join(self.data_dir, "val_red") # Directory in which reduced validation dataset resides
        self.test_red_dir = os.path.join(self.data_dir, "test_red") # Directory in which reduced test dataset resides


    def download_data(self, annotation_format='darknet'):
        # download_dataset(self.data_dir, self.classes, annotation_format=annotation_format, limit=5)
        
        for class_name in self.classes:
            print(f'Attempting to download {class_name} data')
            if not os.path.isdir(os.path.join(self.data_dir, class_name.lower())):
                try:
                    download_dataset(self.data_dir, [class_name], annotation_format=annotation_format, limit=500)
                except Exception as e:
                    print(f'An exception occurred for {class_name}. ERROR: {e}')
            else:
                print(f'Skipped {class_name}, data already downloaded')


    def split_data(self, keep_class_dirs=True):

        """ This function splits the downloaded Open Image dataset, and splits each class into training, validation, and testing sets.
            This function assumes that the required data has already been downloaded."""

        # Setting the random seed:
        random.seed(self.random_seed)
        
        splits = ["train", "val", "test"]

        # Making folders for each of the splits:
        for split in splits:
            split_dir = os.path.join(self.data_dir, split)
            os.makedirs(split_dir, exist_ok=True)

        # Iterating through each class:
        for class_cur in self.classes:
            print(f'Splitting data for class {class_cur}')

            # Getting directories for the images and annotations for each class:
            imgs_dir = os.path.join(self.data_dir, class_cur.lower(), "images")
            anns_dir = os.path.join(self.data_dir, class_cur.lower(), "darknet")

            # Ensuring each class has images and annotations:
            if not imgs_dir:
                raise Exception(f'Images do not exist for {class_cur}!')

            if not anns_dir:
                raise Exception(f'Annotations do not exist for {class_cur}!')

            class_imgs = os.listdir(imgs_dir) # Images for current class
            class_anns = os.listdir(anns_dir) # Annotations for current class
            class_imgs.sort()
            class_anns.sort()

            num_imgs = len(class_imgs) # Number of images and annotations for current class
            
            # Shuffling data:
            inds_list = list(range(num_imgs)) # List of indices ranging for the total number of images
            random.shuffle(inds_list) # Shuffling indices list
            class_imgs = [class_imgs[i] for i in inds_list] # Shuffling class images according to shuffled inds_list
            class_anns = [class_anns[i] for i in inds_list] # Shuffling class annotations according to shuffled inds_list

            ind_train = int(0.8 * num_imgs) # Ending index for the training images
            ind_val = ind_train + int(0.1 * num_imgs) # Ending index for the validation images

            # Splitting images into training, validation, and testing:
            train_imgs = class_imgs[:ind_train]
            val_imgs = class_imgs[ind_train:ind_val]
            test_imgs = class_imgs[ind_val:]

            all_imgs = [train_imgs, val_imgs, test_imgs] # All images
            
            # Splitting annotations into training, validation, and testing:
            train_anns = class_anns[:ind_train]
            val_anns = class_anns[ind_train:ind_val]
            test_anns = class_anns[ind_val:]

            all_anns = [train_anns, val_anns, test_anns] # All annotations
            
            # Looping through all split types and corresponding split images:
            for split_type, split_imgs, split_anns in zip(splits, all_imgs, all_anns):
                if keep_class_dirs:
                    # Creating each split directory for images and annotations for current class:
                    split_dir_img = os.path.join(self.data_dir, split_type, class_cur.lower(), "images")
                    split_dir_ann = os.path.join(self.data_dir, split_type, class_cur.lower(), "annotations")
                else:
                    split_dir_img = os.path.join(self.data_dir, split_type, "images")
                    split_dir_ann = os.path.join(self.data_dir, split_type, "annotations")

                os.makedirs(split_dir_img, exist_ok=True)
                os.makedirs(split_dir_ann, exist_ok=True)

                # Copying each image from initial directory to corresponding split directory for each split:
                for img, ann in zip(split_imgs, split_anns):
                    shutil.copy(os.path.join(imgs_dir, img), os.path.join(split_dir_img, img))
                    shutil.copy(os.path.join(anns_dir, ann), os.path.join(split_dir_ann, ann))

                    # Code to replace each original class label (which is 0) to the class label as found in self.class_2_index:
                    ann_file_cur_dir = os.path.join(split_dir_ann, ann) # File path of current annotation file
                    with open(ann_file_cur_dir, 'r') as file:
                        objects = file.readlines()

                        new_labels = []
                        for obj in objects:
                            obj_items = obj.split()
                            new_class_label = self.class_2_index[class_cur.lower()]
                            obj_items[0] = str(new_class_label)

                            obj_new = ' '.join(obj_items) + '\n'
                            new_labels.append(obj_new)
                    
                    with open(ann_file_cur_dir, 'w') as file:
                        file.writelines(new_labels)
                            

    def split_data_reduced(self, keep_class_dirs=True):

        """ This function splits the downloaded Open Image dataset, and splits each class into training, validation, and testing sets.
            This function assumes that the required data has already been downloaded.
            This function reduces the dataset by self.keep_perc. """

        # Setting the random seed:
        random.seed(self.random_seed)
        
        splits = ["train_red", "val_red", "test_red"]
        
        # Making folders for each of the splits:
        for split in splits:
            split_dir = os.path.join(self.data_dir, split)
            os.makedirs(split_dir, exist_ok=True)

        # Iterating through each class:
        for class_cur in self.classes:
            print(f'Splitting data for class {class_cur}')

            # Getting directories for the images and annotations for each class:
            imgs_dir = os.path.join(self.data_dir, class_cur.lower(), "images")
            anns_dir = os.path.join(self.data_dir, class_cur.lower(), "darknet")

            # Ensuring each class has images and annotations:
            if not imgs_dir:
                raise Exception(f'Images do not exist for {class_cur}!')

            if not anns_dir:
                raise Exception(f'Annotations do not exist for {class_cur}!')

            class_imgs = os.listdir(imgs_dir) # Images for current class
            class_anns = os.listdir(anns_dir) # Annotations for current class
            class_imgs.sort()
            class_anns.sort()

            num_imgs = len(class_imgs) # Number of images and annotations for current class
            
            if self.perc_keep != 1.00 and num_imgs > 50:
                num_imgs = int(num_imgs * self.perc_keep)
                class_imgs = class_imgs[:num_imgs]
                class_anns = class_anns[:num_imgs]

            # Shuffling data:
            inds_list = list(range(num_imgs)) # List of indices ranging for the total number of images
            random.shuffle(inds_list) # Shuffling indices list
            class_imgs = [class_imgs[i] for i in inds_list] # Shuffling class images according to shuffled inds_list
            class_anns = [class_anns[i] for i in inds_list] # Shuffling class annotations according to shuffled inds_list

            ind_train = int(0.8 * num_imgs) # Ending index for the training images
            ind_val = ind_train + int(0.1 * num_imgs) # Ending index for the validation images

            # Splitting images into training, validation, and testing:
            train_imgs = class_imgs[:ind_train]
            val_imgs = class_imgs[ind_train:ind_val]
            test_imgs = class_imgs[ind_val:]

            all_imgs = [train_imgs, val_imgs, test_imgs] # All images
            
            # Splitting annotations into training, validation, and testing:
            train_anns = class_anns[:ind_train]
            val_anns = class_anns[ind_train:ind_val]
            test_anns = class_anns[ind_val:]

            all_anns = [train_anns, val_anns, test_anns] # All annotations
            
            # Looping through all split types and corresponding split images:
            for split_type, split_imgs, split_anns in zip(splits, all_imgs, all_anns):
                if keep_class_dirs:
                    # Creating each split directory for images and annotations for current class:
                    split_dir_img = os.path.join(self.data_dir, split_type, class_cur.lower(), "images")
                    split_dir_ann = os.path.join(self.data_dir, split_type, class_cur.lower(), "annotations")
                else:
                    split_dir_img = os.path.join(self.data_dir, split_type, "images")
                    split_dir_ann = os.path.join(self.data_dir, split_type, "annotations")

                os.makedirs(split_dir_img, exist_ok=True)
                os.makedirs(split_dir_ann, exist_ok=True)

                # Copying each image from initial directory to corresponding split directory for each split:
                for img, ann in zip(split_imgs, split_anns):
                    shutil.copy(os.path.join(imgs_dir, img), os.path.join(split_dir_img, img))
                    shutil.copy(os.path.join(anns_dir, ann), os.path.join(split_dir_ann, ann))

                    # Code to replace each original class label (which is 0) to the class label as found in self.class_2_index:
                    ann_file_cur_dir = os.path.join(split_dir_ann, ann) # File path of current annotation file
                    with open(ann_file_cur_dir, 'r') as file:
                        objects = file.readlines()

                        new_labels = []
                        for obj in objects:
                            obj_items = obj.split()
                            new_class_label = self.class_2_index[class_cur.lower()]
                            obj_items[0] = str(new_class_label)

                            obj_new = ' '.join(obj_items) + '\n'
                            new_labels.append(obj_new)
                    
                    with open(ann_file_cur_dir, 'w') as file:
                        file.writelines(new_labels)

        print(f"Dataset has been reduced!")



    def convert_to_coco(self, reduced=True):
        """ This function strives to convert the darknet annotations into COCO format. """

        # Split names:
        if reduced:
            splits = ["train_red", "val_red", "test_red"]
        else:
            splits = ["train", "val", "test"]

        # Getting categories with category labels and corresponding id's:
        categories = list(self.category_mapping.values())

        # Initializing annotation and image id's:        
        annotation_id = 1
        image_id = 1
        
        # Iterating through all of the splits
        for i, split in enumerate(splits):

            coco_format = {
                "images": [],
                "annotations": [],
                "categories": categories
            }

            img_folder = os.path.join(self.data_dir, split, "images")
            ann_folder = os.path.join(self.data_dir, split, "annotations")

            for img_file in os.listdir(img_folder):
                
                img_file_path = os.path.join(img_folder, img_file)
                img = Image.open(img_file_path)
                width, height = img.size
                coco_format["images"].append({
                    "id": image_id,
                    "file_name": img_file,
                    "width": width,
                    "height": height
                })

                # Annotation:
                ann_file_path = os.path.join(ann_folder, img_file.replace(".jpg", ".txt"))                
                with open(ann_file_path, "r") as f:
                    lines = f.readlines()
                    for line in lines:
                        parts = line.strip().split()
                        class_id = int(parts[0])
                        x_center, y_center, bbox_width, bbox_height = map(float, parts[1:])
                        
                        # Convert to COCO bbox format [x_min, y_min, width, height]:
                        x_min = (x_center - bbox_width / 2) * width
                        y_min = (y_center - bbox_height / 2) * height
                        bbox_width *= width
                        bbox_height *= height
                        
                        # Annotation information:
                        coco_format["annotations"].append({
                            "id": annotation_id,
                            "image_id": image_id,
                            "category_id": class_id,
                            "bbox": [x_min, y_min, bbox_width, bbox_height],
                            "area": bbox_width * bbox_height,
                            "iscrowd": 0
                        })

                        # Incrementing the annotation id:
                        annotation_id += 1
                
                # Incrementing the image id:
                image_id += 1

            # Setting the name of the .json file accordingly:
            if i == 0:
                with open("train_coco.json", "w") as f:
                    json.dump(coco_format, f, indent=4)                    
            elif i == 1:
                with open("val_coco.json", "w") as f:
                    json.dump(coco_format, f, indent=4)                    
            else:
                with open("test_coco.json", "w") as f:
                    json.dump(coco_format, f, indent=4)                    
            
            print(f"Converted {split} to coco format!")




In [8]:
# Data Configuration & Hyperparameters:
PERC_KEEP = 0.10 # Proportion of data from datasets to keep
BATCH_SIZE = 16 # Batch size
EPOCHS = 50 # Number of epochs to train the model for
LEARNING_RATE = 1e-4 # Learning rate

In [19]:
### Loading Open Images Dataset:

# Initializing the DETRDatasetProcessor class:
detr_processor = DETRDatasetProcessor(batch_size=BATCH_SIZE, perc_keep=PERC_KEEP)
print(f"Number of classes: {len(detr_processor.classes)}")

# Downloading the Open Images Dataset in darknet format:
# detr_processor.download_data()

# Splitting the downloaded data into training, validation, and test sets:
# detr_processor.split_data(keep_class_dirs=False)

# Splitting the downloaded data into reduced training, validation, and test sets:
# detr_processor.split_data_reduced(keep_class_dirs=False)

detr_processor.convert_to_coco(reduced=False)



Number of classes: 64
Converted train to coco format!
Converted val to coco format!
Converted test to coco format!


In [12]:
print(detr_processor.category_mapping[63])

{'id': 64, 'name': 'sushi'}


In [ ]:
# Can also run this command in the terminal:
%run detr/main.py --coco_path ".\data\detr" --output_dir "." --epochs 150

Not using distributed mode
git:
  sha: N/A, status: clean, branch: N/A

Namespace(lr=0.0001, lr_backbone=1e-05, batch_size=2, weight_decay=0.0001, epochs=150, lr_drop=200, clip_max_norm=0.1, frozen_weights=None, backbone='resnet50', dilation=False, position_embedding='sine', enc_layers=6, dec_layers=6, dim_feedforward=2048, hidden_dim=256, dropout=0.1, nheads=8, num_queries=100, pre_norm=False, masks=False, aux_loss=True, set_cost_class=1, set_cost_bbox=5, set_cost_giou=2, mask_loss_coef=1, dice_loss_coef=1, bbox_loss_coef=5, giou_loss_coef=2, eos_coef=0.1, dataset_file='coco', coco_path='.\\data\\detr', coco_panoptic_path=None, remove_difficult=False, output_dir='.', device='cuda', seed=42, resume='', start_epoch=0, eval=False, num_workers=2, world_size=1, dist_url='env://', distributed=False)


KeyboardInterrupt: 

In [ ]:
# Loading pre-trained DETR R50 model:
checkpoint_path = "detr-r50-e632da11.pth"  # Path to the original checkpoint
checkpoint = torch.load(checkpoint_path, map_location="cpu")

# Removing the class_embed weights from the checkpoint:
checkpoint["model"] = {k: v for k, v in checkpoint["model"].items() if not k.startswith("class_embed")}

# Loading the pretrained DETR model from PyTorch Hub to have a base model object to work off of:
detr_model = torch.hub.load('facebookresearch/detr', 'detr_resnet50', pretrained=True)

# Replacing the classification head to match the number of custom classes:
num_labels = 66
detr_model.class_embed = nn.Linear(detr_model.class_embed.in_features, num_labels)

# Initializing new classification head:
nn.init.xavier_uniform_(detr_model.class_embed.weight)
nn.init.constant_(detr_model.class_embed.bias, 0)

# Loading the modified checkpoint into the model:
detr_model.load_state_dict(checkpoint["model"], strict=False)

# Saving the modified model's state dictionary:
model_path = "modified_detr.pth"
torch.save({'model': detr_model.state_dict()}, model_path)

print(f"Modified model saved to {model_path}")


Using cache found in C:\Users\prave/.cache\torch\hub\facebookresearch_detr_main


Modified model saved to modified_detr.pth
